## Hipótese 1
### Quais lojas estão com dias de ruptura maior que o leadtime?


In [9]:
import pandas as pd
import numpy as np
import os
import datetime

# Froms
from gcpUtils.auth import getCredentials
from gcpUtils.bigQuery import pandasToBq, tableToPandas
from gcpUtils.google_storage_manager import *

# Definição das credenciais 
cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

## Rupturas em Geral
* Filtramos o banco de dado e tomamos apenas os dados relativos a filiais com CM no final;
* Estabelecemos um período de um ano (facilmente alterável)
* Contamos os dias em que o produto ficou em ruptura, desconsiderando buracos problemáticos, isto é, tomamos por finalização do estado de ruptura o momento em que uma mudança de 1 para 0, mesmo que haja um buraco de dados anterior a isso.
* Ao final, ordenamos por filial e SKU. 

#### Registramos um ID para cada período de ruptura, registrando também a quantidade de dias relativos a cada um deles. Exemplo: se houve $n$ estados de ruptura relativos ao SKU='X', então há $1,2,3,...n$ IDs diferentes, onde cada um indica um estado de ruptura registrado.
#### Os dados então são salvos na pasta "data", em um arquivo xlsx.

# 1 - Calcular dias de ruptura

In [10]:
query = """
WITH rupturas AS (
    SELECT
        FILIAL,
        SKU,
        DATA,
        CONT_RUPTURA,
        LEADTIME,
        CASE 
            WHEN CONT_RUPTURA = 1
                AND LAG(CONT_RUPTURA, 1, 0) OVER (
                    PARTITION BY FILIAL, SKU ORDER BY DATA
                ) = 0
            THEN 1 ELSE 0
        END AS inicio_ruptura
    FROM `planejamento-animale-292719.checklists_rollout.ANIMALE_checklist`
    WHERE DATA > '2024-08-01' 
      AND DATA < '2025-09-01'
      AND FILIAL LIKE '%CM%' 
),
grupos AS (
    SELECT
        FILIAL,
        SKU,
        DATA,
        CONT_RUPTURA,
        LEADTIME,
        SUM(inicio_ruptura) OVER (
            PARTITION BY FILIAL, SKU ORDER BY DATA ROWS UNBOUNDED PRECEDING
        ) AS RUPTURA_ID
    FROM rupturas
    WHERE CONT_RUPTURA = 1
),
resumo AS (
    SELECT
        FILIAL,
        SKU,
        RUPTURA_ID,
        MIN(CAST(DATA AS DATE)) AS inicio_ruptura,
        MAX(CAST(DATA AS DATE)) AS fim_ruptura,
        DATE_DIFF(
        MAX(CAST(DATA AS DATE)), MIN(CAST(DATA AS DATE)), DAY
        ) + 1 AS DIAS_RUPTURA,
        MAX(LEADTIME) AS LEADTIME,
    FROM grupos
    GROUP BY FILIAL, SKU, RUPTURA_ID
)
SELECT *
FROM resumo
ORDER BY FILIAL, SKU, RUPTURA_ID;

"""

In [11]:
df_prod_desc = tableToPandas(query, 'planejamento-animale-292719', cred)
df_prod_desc.columns = df_prod_desc.columns.str.upper()

# print(df_prod_desc)

In [12]:
# Salvando o dataframe em um arquivo excell localmente
local_path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(local_path):
    os.makedirs(local_path)
file_path = os.path.join(local_path, 'rupturas_anuale_checklist.xlsx')
df_prod_desc.to_excel(file_path, index=False)


# 2 - Comparar com os leadtime

In [ ]:
conditions = [
    df_prod_desc['LEADTIME'] > df_prod_desc['DIAS_RUPTURA'],
    df_prod_desc['LEADTIME'] == df_prod_desc['DIAS_RUPTURA'],
    df_prod_desc['LEADTIME'] < df_prod_desc['DIAS_RUPTURA']
]
choices = ["Bom", "Razoável", "Ruim"]

# criar nova coluna 'AVALIACAO' com base nas condições
df_prod_desc['AVALIACAO'] = ''
df_prod_desc['AVALIACAO'] = np.select(conditions, choices, default='Desconhecido')

local_path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(local_path):
    os.makedirs(local_path)

file_path = os.path.join(local_path, 'rupturas_avaliadas_checklist.xlsx')
df_prod_desc.to_excel(file_path, index=False)

# 3 - Implementação do IET

In [8]:
# IET = (LEADTIME-DIASRUPTURA)/(LEADTIME+DIASRUPTURA)
df_prod_desc['IET'] = (df_prod_desc['LEADTIME'] - df_prod_desc['DIAS_RUPTURA']) / (df_prod_desc['LEADTIME'] + df_prod_desc['DIAS_RUPTURA'])
file_path = os.path.join(local_path, 'rupturas_avaliadas_iet_checklist.xlsx')
df_prod_desc.to_excel(file_path, index=False)